In [1]:
from stockmarket.limitorderbook import *
from stockmarket.agent import Trader
from stockmarket.firms import Firm
from stockmarket.stock import Stock

In [32]:
from numpy.testing import assert_equal
from numpy.testing import assert_raises
import operator
from stockmarket import functions, setup, marketmechanisms, randomset, database
import random

## To test
1. setup orderbook
2. run model

## setup orderbook

In [49]:
seed=1
simulation_time=100
amount_fundamentalists=40
amount_chartists=40
amount_firms=3
initial_money=(100, 200)
initial_bid_ask=(0,0)
initial_memory=(3,7)
initial_ma_short=(1,3)
initial_ma_long=(6,7)
initial_profit=(200,200)
initial_book_value=(10000, 10000)
initial_stock_amount=400
observable_set_size=3

In [50]:
agents = setup.setup_agents(init_money=initial_money,
                                init_bid_ask_spread=initial_bid_ask,
                                init_memory_size=initial_memory,
                                init_ma_s=initial_ma_short,
                                init_ma_l=initial_ma_long,
                                fundamentalist=amount_fundamentalists,
                                chartist=amount_chartists)

firms = setup.setup_firms(init_book_value=initial_book_value,
                              init_profit=initial_profit,
                              amount_of_firms=amount_firms)

stocks = setup.setup_stocks(firms, amount=initial_stock_amount)

order_books = []
for stock in stocks:
    order_books.append(LimitOrderBook(stock, 100, 120))

setup.distribute_initial_stocks(stocks, agents)

In [39]:
order_books[2]

order_book_Stock of 2

### 1 create new limitorderBook

In [3]:
# create a firm
firm = Firm("Firm1", 10000, [200, 300, 400, 300], 1)
# create a stock of that firm
stocks = Stock(firm, 1000)

In [4]:
# create new limitOrderBook
order_book = LimitOrderBook(stocks, 100, 120)

In [5]:
order_book

order_book_Stock of Firm1

### 2 add bids to the limit orderbook
* The highest bid should be at the end of the list.
* For bids of equal value, the oldest bid should be at the top

In [6]:
order_book.add_bid(10, 20, 'trader-1')
order_book.bids

[Order_p=10_t=b_o=trader-1_a=0]

In [7]:
order_book.add_bid(5, 20, 'trader-2')
order_book.bids

[Order_p=5_t=b_o=trader-2_a=0, Order_p=10_t=b_o=trader-1_a=0]

In [8]:
order_book.add_bid(7, 20, 'trader-3')
order_book.bids

[Order_p=5_t=b_o=trader-2_a=0,
 Order_p=7_t=b_o=trader-3_a=0,
 Order_p=10_t=b_o=trader-1_a=0]

In [9]:
order_book.add_bid(7, 20, 'trader-4')
order_book.bids

[Order_p=5_t=b_o=trader-2_a=0,
 Order_p=7_t=b_o=trader-4_a=0,
 Order_p=7_t=b_o=trader-3_a=0,
 Order_p=10_t=b_o=trader-1_a=0]

In [10]:
order_book.add_bid(7, 20, 'trader-5')
order_book.bids

[Order_p=5_t=b_o=trader-2_a=0,
 Order_p=7_t=b_o=trader-5_a=0,
 Order_p=7_t=b_o=trader-4_a=0,
 Order_p=7_t=b_o=trader-3_a=0,
 Order_p=10_t=b_o=trader-1_a=0]

In [11]:
order_book.bids[0].price

5

In [12]:
assert_equal(order_book.bids[0].price, 5)
assert_equal(order_book.bids[-1].price, 10)

In [13]:
assert_equal(order_book.bids[-2].owner, 'trader-3')

### 3 Add aks to the limit order book
* The lowest ask should be at the beginning of the list.
* For asks of equal value, the oldest ask should be at the bottom

In [14]:
order_book.add_ask(11, 20, 'trader-1')
order_book.asks

[Order_p=11_t=a_o=trader-1_a=0]

In [15]:
order_book.add_ask(5, 20, 'trader-2')
order_book.add_ask(7, 20, 'trader-3')
order_book.add_ask(7, 20, 'trader-4')
order_book.add_ask(7, 20, 'trader-5')
order_book.asks

[Order_p=5_t=a_o=trader-2_a=0,
 Order_p=7_t=a_o=trader-3_a=0,
 Order_p=7_t=a_o=trader-4_a=0,
 Order_p=7_t=a_o=trader-5_a=0,
 Order_p=11_t=a_o=trader-1_a=0]

In [16]:
# highest ask is 11
assert_equal(order_book.asks[-1].price, 11)
# lowest ask is 5
assert_equal(order_book.asks[0].price, 5)
# second highest ask trader = trader-5
assert_equal(order_book.asks[-2].owner, 'trader-5')

### 4 clean aging bids and asks

In [54]:
import numpy as np

In [60]:
np.isfinite(np.inf)

False

In [51]:
for n in range(120):
    order_book.clean_book()

In [52]:
order_book.add_bid(10, 20, 'trader-1')
order_book.add_bid(5, 20, 'trader-2')
order_book.add_ask(11, 20, 'trader-1')
order_book.add_ask(5, 20, 'trader-2')
matched_orders = 0
while True:
    matched_orders = order_book.match_orders()
    if matched_orders is None:
        print('i am getting out')
        break
    print(matched_orders)

(5, 20, Order_p=10_t=b_o=trader-1_a=0, Order_p=5_t=a_o=trader-2_a=0)
i am getting out


In [46]:
print(matched_orders)

None


In [18]:
order_book.add_bid(10, 20, 'trader-1')
order_book.add_bid(5, 20, 'trader-2')
order_book.add_ask(11, 20, 'trader-1')
order_book.add_ask(5, 20, 'trader-2')
for n in range(119):
    order_book.clean_book()
assert_equal(len(order_book.bids), 2)
assert_equal(len(order_book.asks), 2)
order_book.clean_book()
assert_equal(len(order_book.bids), 0)
assert_equal(len(order_book.asks), 0)

### 5 match the best bid with the best ask and remove them 

### Test if initial orders are properly dealt with

In [28]:
# add some asks
order_book.add_ask(5, 20, 'trader-2')
order_book.add_ask(7, 20, 'trader-3')
order_book.add_ask(7, 20, 'trader-4')
order_book.add_ask(7, 20, 'trader-5')
# and bids
order_book.add_bid(10, 20, 'trader-1')
order_book.add_bid(4, 20, 'trader-11')
order_book.add_bid(9, 20, 'trader-11')
matched_orders = order_book.match_orders()
# after an orderbook match both order books are reduced by 1
assert_equal(len(order_book.bids), 2)
assert_equal(len(order_book.asks), 3)
# and the highest bid was matched to the lowest ask, difference is 5
assert_equal(matched_orders[2].price - matched_orders[3].price, 5)
# the second match is bid p9 and ask p7, difference is 2
matched_orders = order_book.match_orders()
assert_equal(matched_orders[2].price - matched_orders[3].price, 2)
# Once again the order books are reduced by 1 in size
assert_equal(len(order_book.bids), 1)
assert_equal(len(order_book.asks), 2)
# then no more match is possible 
matched_orders = order_book.match_orders()
assert_equal(matched_orders, None)
for n in range(500):
    order_book.clean_book()
order_book.add_ask(5, 10, 'trader-2')
order_book.add_ask(7, 8, 'trader-3')
order_book.add_bid(10, 20, 'trader-1')
# first match should reduce asks book by 1
matched_orders = order_book.match_orders()
assert_equal(len(order_book.bids), 1)
assert_equal(len(order_book.asks), 1)
assert_equal(matched_orders[2].price - matched_orders[3].price, 5)
# the bid should have a remaining volume of 10
assert_equal(order_book.bids[0].volume, 10)
# second match 
matched_orders = order_book.match_orders()
# should reduce the lenght of the asks book to zero and bids should remain 1
assert_equal(len(order_book.bids), 1)
assert_equal(len(order_book.asks), 0)
# the bid should have a remaining volume of 2
assert_equal(order_book.bids[0].volume, 2)
# price should be 7 and volume 8
assert_equal(matched_orders[0], 7)
assert_equal(matched_orders[1], 8)
# no more matches should be possible, leaving the order in the orderbook
assert_equal(order_book.match_orders(), None)

In [27]:
# clean order book
for n in range(500):
    order_book.clean_book()

# 2 Implement in model 

In [ ]:
# reduce both bid and ask by volume